In [1]:
import os
import dlib
import cv2
import numpy as np
import shutil

In [11]:
img_path = 'C:/Users/jungh/final_project/img_path/' # 이미지가 저장된 폴더 경로

crop_path = 'C:/Users/jungh/final_project/img_path/crop_path' # 얼굴 crop된 이미지를 저장할 폴더 경로

eye_path = 'C:/Users/jungh/final_project/img_path/eye_path' # 눈 영역이 저장될 폴더 경로

datFile =  'C:/Users/jungh/final_project/landmarks/shape_predictor_68_face_landmarks.dat'

# Dlib의 얼굴 인식기 초기화
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(datFile)



In [12]:
folder_list = os.listdir(img_path)

In [13]:

# 각 폴더에 대해 실행
for folder in folder_list:
    folder_path = os.path.join(img_path, folder)
    if os.path.isdir(folder_path):
        # crop_path의 동일한 폴더 생성
        crop_folder_path = os.path.join(crop_path, folder)
        os.makedirs(crop_folder_path, exist_ok=True)
        
        # eye_path의 동일한 폴더 생성
        eye_folder_path = os.path.join(eye_path, folder)
        os.makedirs(eye_folder_path, exist_ok=True)
        
        # 서브폴더 목록 가져오기
        subfolder_list = os.listdir(folder_path)
        
        # 각 서브폴더에 대해 실행
        for subfolder in subfolder_list:
            subfolder_path = os.path.join(folder_path, subfolder)
            if os.path.isdir(subfolder_path):
                # crop_path의 동일한 서브폴더 생성
                crop_subfolder_path = os.path.join(crop_folder_path, subfolder)
                os.makedirs(crop_subfolder_path, exist_ok=True)
                
                # eye_path의 동일한 서브폴더 생성
                eye_subfolder_path = os.path.join(eye_folder_path, subfolder)
                os.makedirs(eye_subfolder_path, exist_ok=True)
                
                # 서브폴더 내의 이미지 파일 목록 가져오기
                image_list = os.listdir(subfolder_path)
                
                # 각 이미지에 대해 실행
                for image_name in image_list:
                    image_path = os.path.join(subfolder_path, image_name)
                    
                    try:
                        # 이미지 불러오기
                        image = cv2.imread(image_path)
                        
                        # 이미지를 흑백으로 변환 (얼굴의 위치를 찾는데 색상 정보는 필요하지 않기 때문에 인식된 프레임을 흑백 이미지로 변환 후 detector에 전달)
                        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                        
                        # 얼굴 인식
                        faces = detector(gray)
                        
                        # 각 얼굴에 대해 실행
                        for face in faces:
                            # 얼굴의 랜드마크 추출
                            landmarks = predictor(gray, face)
                            
                            # 눈 영역 추출
                            left_eye_points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
                            right_eye_points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])
                            
                            # 눈 영역 다각형 생성
                            left_eye_mask = np.zeros_like(image)
                            cv2.fillPoly(left_eye_mask, [left_eye_points], (255, 255, 255))
                            left_eye_masked = cv2.bitwise_and(image, left_eye_mask)
                            left_eye_rect = cv2.boundingRect(left_eye_points)
                            left_eye_cropped = left_eye_masked[left_eye_rect[1]:left_eye_rect[1] + left_eye_rect[3],
                                               left_eye_rect[0]:left_eye_rect[0] + left_eye_rect[2]]
                            
                            # 오른쪽도 만들긴 했지만 한쪽만 있으면 될것 같아서 오른쪽 눈은 따로 저장 안함
                            right_eye_mask = np.zeros_like(image)
                            cv2.fillPoly(right_eye_mask, [right_eye_points], (255, 255, 255))
                            right_eye_masked = cv2.bitwise_and(image, right_eye_mask)
                            right_eye_rect = cv2.boundingRect(right_eye_points)
                            right_eye_cropped = right_eye_masked[right_eye_rect[1]:right_eye_rect[1] + right_eye_rect[3],
                                                right_eye_rect[0]:right_eye_rect[0] + right_eye_rect[2]]
                            
                            # 눈 영역 추출 후 저장
                            eye_image_path = os.path.join(eye_subfolder_path, image_name)
                            cv2.imwrite(eye_image_path, left_eye_cropped)
                            
                            # 얼굴 crop 후 저장
                            crop_image = image[face.top():face.bottom(), face.left():face.right()]
                            crop_image_path = os.path.join(crop_subfolder_path, image_name)
                            cv2.imwrite(crop_image_path, crop_image)
                    
                    # 코드를 실행 시켰을때 사진을 인식 못해서 멈추는 경우가 있어서 어떤 파일이 오류가 났는지 확인하기위해 try except를 사용
                    except Exception as e:
                        print(f"오류 발생: {image_name} - {str(e)}")
                        continue

오류 발생: de7cef7dd3fae5944db2f70dc4a46f34.jpg - OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

오류 발생: 2.jpg - OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

오류 발생: ceba53ef8e9d79c2662e23f8fc3f6253.jpg - OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

오류 발생: 3.jpg - OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

오류 발생: 23.jpg - OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

오류 발생: ec5e95593d7df8a1e10f90326

# 저장된 사진 분류하기 위해 사용

In [10]:
# 이미지가 저장된 폴더 경로
img_path = 'C:/Users/jungh/final_project/img_path/'

# crop_path: 얼굴 crop된 이미지가 저장된 폴더 경로
crop_path = 'C:/Users/jungh/final_project/img_path/crop_path1/'

# eye_path: 눈 영역이 저장된 폴더 경로
eye_path = 'C:/Users/jungh/final_project/img_path/eye_path1/'

# cool 폴더 내의 이미지 처리
cool_path = os.path.join(crop_path, "cool")
if os.path.exists(cool_path):
    # winter 폴더 생성
    winter_path = os.path.join(crop_path, "winter")
    os.makedirs(winter_path, exist_ok=True)
    
    # summer 폴더 생성
    summer_path = os.path.join(crop_path, "summer")
    os.makedirs(summer_path, exist_ok=True)
    
    # cool 폴더 내의 하위폴더 처리
    for folder_name in os.listdir(cool_path):
        folder_path = os.path.join(cool_path, folder_name)
        if os.path.isdir(folder_path):
            # winter 폴더로 이동
            if folder_name.startswith("winter"):
                target_folder = winter_path
            # summer 폴더로 이동
            elif folder_name.startswith("summer"):
                target_folder = summer_path
            else:
                continue
            
            # 하위폴더 내의 파일 처리
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                
                # 중복된 파일 이름 수정
                target_path = os.path.join(target_folder, file_name)
                counter = 10
                while os.path.exists(target_path):
                    file_name_parts = os.path.splitext(file_name)
                    base_name = file_name_parts[0]
                    extension = file_name_parts[1]
                    new_file_name = f"{base_name}_{counter}{extension}"
                    target_path = os.path.join(target_folder, new_file_name)
                    counter += 1
                
                shutil.move(file_path, target_path)

# warm 폴더 내의 이미지 처리
warm_path = os.path.join(crop_path, "warm")
if os.path.exists(warm_path):
    # spring 폴더 생성
    spring_path = os.path.join(crop_path, "spring")
    os.makedirs(spring_path, exist_ok=True)
    
    # fall 폴더 생성
    fall_path = os.path.join(crop_path, "fall")
    os.makedirs(fall_path, exist_ok=True)
    
    # warm 폴더 내의 하위폴더 처리
    for folder_name in os.listdir(warm_path):
        folder_path = os.path.join(warm_path, folder_name)
        if os.path.isdir(folder_path):
            # spring 폴더로 이동
            if folder_name.startswith("spring"):
                target_folder = spring_path
            # fall 폴더로 이동
            elif folder_name.startswith("fall"):
                target_folder = fall_path
            else:
                continue
            
            # 하위폴더 내의 파일 처리
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                
                # 중복된 파일 이름 수정
                target_path = os.path.join(target_folder, file_name)
                counter = 22
                while os.path.exists(target_path):
                    file_name_parts = os.path.splitext(file_name)
                    base_name = file_name_parts[0]
                    extension = file_name_parts[1]
                    new_file_name = f"{base_name}_{counter}{extension}"
                    target_path = os.path.join(target_folder, new_file_name)
                    counter += 1
                
                shutil.move(file_path, target_path)

In [9]:
# cool 폴더 내의 이미지 처리
cool_path = os.path.join(eye_path, "cool")
if os.path.exists(cool_path):
    # winter 폴더 생성
    winter_path = os.path.join(eye_path, "winter")
    os.makedirs(winter_path, exist_ok=True)
    
    # summer 폴더 생성
    summer_path = os.path.join(eye_path, "summer")
    os.makedirs(summer_path, exist_ok=True)
    
    # cool 폴더 내의 하위폴더 처리
    for folder_name in os.listdir(cool_path):
        folder_path = os.path.join(cool_path, folder_name)
        if os.path.isdir(folder_path):
            # winter 폴더로 이동
            if folder_name.startswith("winter"):
                target_folder = winter_path
            # summer 폴더로 이동
            elif folder_name.startswith("summer"):
                target_folder = summer_path
            else:
                continue
            
            # 하위폴더 내의 파일 처리
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                
                # 중복된 파일 이름 수정
                target_path = os.path.join(target_folder, file_name)
                counter = 10
                while os.path.exists(target_path):
                    file_name_parts = os.path.splitext(file_name)
                    base_name = file_name_parts[0]
                    extension = file_name_parts[1]
                    new_file_name = f"{base_name}_{counter}{extension}"
                    target_path = os.path.join(target_folder, new_file_name)
                    counter += 1
                
                shutil.move(file_path, target_path)

# warm 폴더 내의 이미지 처리
warm_path = os.path.join(eye_path, "warm")
if os.path.exists(warm_path):
    # spring 폴더 생성
    spring_path = os.path.join(eye_path, "spring")
    os.makedirs(spring_path, exist_ok=True)
    
    # fall 폴더 생성
    fall_path = os.path.join(eye_path, "fall")
    os.makedirs(fall_path, exist_ok=True)
    
    # warm 폴더 내의 하위폴더 처리
    for folder_name in os.listdir(warm_path):
        folder_path = os.path.join(warm_path, folder_name)
        if os.path.isdir(folder_path):
            # spring 폴더로 이동
            if folder_name.startswith("spring"):
                target_folder = spring_path
            # fall 폴더로 이동
            elif folder_name.startswith("fall"):
                target_folder = fall_path
            else:
                continue
            
            # 하위폴더 내의 파일 처리
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                
                # 중복된 파일 이름 수정
                target_path = os.path.join(target_folder, file_name)
                counter = 1
                while os.path.exists(target_path):
                    file_name_parts = os.path.splitext(file_name)
                    base_name = file_name_parts[0]
                    extension = file_name_parts[1]
                    new_file_name = f"{base_name}_{counter}{extension}"
                    target_path = os.path.join(target_folder, new_file_name)
                    counter += 1
                
                shutil.move(file_path, target_path)